In [ ]:
### Autodock preparation ### from: https://www.chem.uwec.edu/chem491_w01/Chem491-Molecules%20and%20Medicine%202008/AutoDock%20Tutorial.pdf

In [ ]:


from SBLMDCOVDOCK.SBLSettings import DOCKING_Settings
from SBLMDCOVDOCK.md_setuptools import plot_xvg
import pickle
from copy import deepcopy
import subprocess

import pandas as pd
import os
import glob

# import gmxapi as gmx
import subprocess
import matplotlib.pyplot as plt

settings = DOCKING_Settings()
main_dir = os.getcwd()

# AMPC - https://journals.asm.org/doi/10.1128/AAC.02073-20
ampc = "6T3D"
# KPC-2 - BLDB: http://dx.doi.org/10.1021/ACS.JMEDCHEM.7B00158
kpc2 = "5UL8"
# OXA-10 - BLDB: https://www.pnas.org/doi/full/10.1073/pnas.241442898
oxa10 = "1K55"

structures = pd.DataFrame({"PDBID": [ampc, kpc2, oxa10], 
                           "Name": ["AmpC", "KPC2", "OXA10"],
                           "Cat_Ser": [64,70,67]})



In [ ]:
prepare_ligand_path = os.path.join(settings.structures_input, "prepare_ligand4.py")
print(prepare_ligand_path)
print(os.path.exists(prepare_ligand_path))
print(main_dir)
os.chdir(main_dir)

In [ ]:
ligands = os.listdir(settings.structures_input)
ligands = [ligand for ligand in ligands if ligand.endswith(".sdf")]
print(ligands)

In [ ]:
dir = "/home/alexi/Documents/SBL_MD_CovDock/topology/Dock"


In [ ]:
pythonsh= "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh"
print(dir)
for pdb in structures.PDBID:
    script_name = 'prepare_receptor4.py'
    prepare_rec_path = os.path.join("..","..",os.path.join(settings.config, settings.parent, script_name))
    pdb_file = f"APO_{pdb}_chimeraFH.pdb"

    out_pdb_name = "APO_" +pdb + "_ADT" + ".pdb"
    pdbqt_name = out_pdb_name.replace(".pdb", ".pdbqt")

    prepare_receptor_command = [pythonsh, prepare_rec_path,
                                "-r", pdb_file,
                                "-o", pdbqt_name,
                                # "-A", "checkhydrogens",
                                "-e", 
                                "-v",

                                ]

    print(" ".join(prepare_receptor_command))
    subprocess.run(prepare_receptor_command, cwd = dir,check=True)




In [ ]:
os.getcwd()

In [ ]:
import subprocess
pythonsh= "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh"
# os.chdir(settings.structures_input)

for idx, lig in enumerate(ligands):
    lig_pdb = lig.replace(".sdf", ".pdb")
    print(idx, lig, lig_pdb)
    obabel_command = ["obabel", 
                      "-i", "sdf", 
                      "-o", "pdb", lig, 
                      "-O", lig_pdb, 
                    #   "-h"
                      ]

    subprocess.run(obabel_command, 
                       check=True,
                       cwd=settings.structures_input)

    # break
# os.chdir(main_dir)


In [ ]:
os.listdir(dir)

In [ ]:
import subprocess
pythonsh= "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh"
# os.chdir(settings.structures_input)
ligs = []
for idx, lig in enumerate(ligands):
    lig_pdb = lig.replace(".sdf", ".pdb")

    script = "prepare_ligand4.py"

    lig_out = lig_pdb.replace(".pdb", ".pdbqt")

    lig_pdb = os.path.join(settings.structures_input, lig_pdb)
    lig_out = os.path.join(settings.topology,settings.parent, lig_out)    

    print(idx, lig_pdb, lig_out)
    script_path = os.path.join(settings.structures_input, script)
    preplig4_command = [pythonsh, script_path, 
                        "-l", lig_pdb, 
                        "-o", lig_out,
                        "-A", "bonds_hydrogens",
                        "-U", "nphs_lps",
                        "-v"]
    try:
        subprocess.run(preplig4_command, 
                       check=True)
        ligs.append(lig.replace(".sdf", ""))
    except:
        pass

    # break
# os.chdir(main_dir)


In [ ]:
print(ligs)

In [ ]:
pythonsh= "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh"

for lig in ligs:
    for pdb in structures.PDBID:
        # break
        print(os.getcwd())
        script_name = "prepare_gpf4.py"
        prepare_grid_path = os.path.join("..","..",os.path.join(settings.config, settings.parent, script_name))

        pdb_name = "APO_" +pdb + "_ADT" + ".pdb"
        pdbqt_name = pdb_name.replace(".pdb", ".pdbqt")
        ligand_name = lig + ".pdbqt"
        prep_grid_command = [pythonsh,
                            prepare_grid_path,
                            "-r", pdbqt_name,
                            "-l", ligand_name,
                            "-o", lig +"_"+ pdbqt_name.replace(".pdbqt", ".gpf"),
                            # "-p", "ligand_types='H,HD,HS,C,A,N,NA,NS,OA,OS,F,Mg,Mn,Zn,Ca,Fe,Cl,Br,I,SA,S,Se,P,GA,GC,GN,GO,GP,G'",
                            
                            "-v"]
        
        print(" ".join(prep_grid_command))
        dir = "/home/alexi/Documents/SBL_MD_CovDock/topology/Dock"

        subprocess.run(prep_grid_command, cwd=dir)


    # break

In [ ]:
pythonsh= "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh"

for lig in ligs:
    for pdb in structures.PDBID:

        print(os.getcwd())
        script_name = "prepare_dpf4.py"
        prepare_dock_path = os.path.join("..","..",os.path.join(settings.config, settings.parent, script_name))

        pdb_name = "APO_" +pdb + "_ADT" + ".pdb"
        pdbqt_name = pdb_name.replace(".pdb", ".pdbqt")
        ligand_name = lig + ".pdbqt"
        prep_dock_command = [pythonsh,
                            prepare_dock_path,
                            "-l", ligand_name,
                            "-r", pdbqt_name,
                            "-o", lig +"_"+ pdbqt_name.replace(".pdbqt", ".dpf"),
                            "-v"]

        print(" ".join(prep_dock_command))
        dir = "/home/alexi/Documents/SBL_MD_CovDock/topology/Dock"

        subprocess.run(prep_dock_command, cwd=dir, check=True)




In [ ]:
print(os.getcwd())

In [ ]:
print(os.listdir(dir))

In [ ]:

autodockgpu_dir = "/home/alexi/Documents/AutoDock-GPU/bin"
engine = "CUDA_64"

autodockgpu = f"autodock_{engine}wi"
auto_dock = os.path.join(autodockgpu_dir, autodockgpu)
docked = []
# for pdb in structures.PDBID:
pdb="5UL8"
for lig in ligs:
# for pdb in structures.PDBID:
    # print("Docking ", lig, " to ", pdb)
    name = "APO_" +pdb + "_ADT"
    pdb_name =  name + ".pdb"
    pdbqt_name = pdb_name.replace(".pdb", ".pdbqt")
    grid_name = lig +"_"+ pdbqt_name.replace(".pdbqt", ".gpf")
    grid_log = grid_name.replace(".gpf", ".glg")

    autogrid_command = ["autogrid4", 
                        "-p", grid_name, 
                        "-l", grid_log, 
                        "-d", f"{lig}_{name}.maps.fld",
                        "-o", f"{lig}_{name}.maps.map",
                        "-v"]

    dock_name = lig +"_"+ pdbqt_name.replace(".pdbqt", ".dpf")
    dock_log = dock_name.replace(".dpf", ".dlg")

    autodock_command = [auto_dock, 
                        # "-p", dock_name, 
                        # "-l", dock_log, 
                        "-M",  f"{lig}_{name}.maps.fld", 
                        "-L", f"{lig}.pdbqt",
                        "--lsmet", "ad",
                        # "--psize", "50",
                        # "--nrun", "50",
                        "-v"]


    dir = "/home/alexi/Documents/SBL_MD_CovDock/topology/Dock"
    # print(os.listdir(dir))
    try:
        subprocess.run(autogrid_command, check=True,cwd=dir)
        subprocess.run(autodock_command, check=True,cwd=dir)
        print("Docking successful")
        docked.append((lig, pdb))
    except:
        pass
    # break
print(docked)

In [ ]:
print(docked)